## Dự tính kết quả đạt được:
**$xy^2z$** sẽ là số lượng data tối đa có thể đạt được trong pipeline, với:

- **$x$** là số context
- **$y$** là số lượng các Llm tham gia vào pipeline
- **$z$** là số question được tạo ra với mỗi Llm

Ví dụ: với **$x = 2790$**, **$y = 7$** và **$z = 7$** sẽ cho ra khoảng **956.000 data** (chưa qua sàn lọc).


In [1]:
import pandas as pd
import ast

## Đọc dataframe chứa các context về dịch vụ công

In [62]:
df = pd.read_parquet('./data/corpus.parquet')
print('Size of context_df: ',len(df))

Size of context_df:  2790


In [5]:
df.head()

,doc_id,contents,metadata
0,b4597feb-b61e-405d-b270-89b3bba0706f,Chi tiết thủ tục hành chính:\r\nMã thủ tục:\r\...,"{'creation_date': '2024-07-17', 'file_name': '..."
1,62513271-21bc-4659-b84a-afcf4549f8c4,Chi tiết thủ tục hành chính:\r\nMã thủ tục:\r\...,"{'creation_date': '2024-07-17', 'file_name': '..."
2,95e087d4-5774-403d-82ed-89c1623a331f,Chi tiết thủ tục hành chính:\r\nMã thủ tục:\r\...,"{'creation_date': '2024-07-17', 'file_name': '..."
3,61d12ba1-2398-45be-9dcc-928b2ed58068,Chi tiết thủ tục hành chính:\r\nMã thủ tục:\r\...,"{'creation_date': '2024-07-17', 'file_name': '..."
4,b439bce4-2cb8-48fb-957a-95ea40bcf724,Chi tiết thủ tục hành chính:\r\nMã thủ tục:\r\...,"{'creation_date': '2024-07-17', 'file_name': '..."


In [9]:
df['contents'][0].split('\n')

['Chi tiết thủ tục hành chính:\r',
 'Mã thủ tục:\r',
 '1.000009\r',
 'Số quyết định:\r',
 '818 /QĐ-BNN-TCLN\r',
 'Tên thủ tục:\r',
 'Cấp ấn phẩm Chứng chỉ CITES xuất khẩu mẫu vật lưu niệm\r',
 'Cấp thực hiện:\r',
 'Cấp Bộ\r',
 'Loại thủ tục:\r',
 'TTHC được luật giao quy định chi tiết\r',
 'Lĩnh vực:\r',
 'Kiểm lâm\r',
 'Trình tự thực hiện:\r',
 'Tổ chức, cá nhân có nhu cầu được cấp ấn phẩm Chứng chỉ CITES xuất khẩu mẫu vật lưu niệm gửi 01 hồ sơ tới Cơ quan thẩm quyền quản lý CITES Việt Nam.\r',
 'Trong thời hạn 05 ngày làm việc, kể từ ngày nhận được hồ sơ hợp lệ, Cơ quan thẩm quyền quản lý CITES Việt Nam có trách nhiệm cấp ấn phẩm Chứng chỉ CITES xuất khẩu mẫu vật lưu niệm cho tổ chức, cá nhân.\r',
 'Trường hợp hồ sơ không hợp lệ, trong thời hạn 03 ngày làm việc, kể từ ngày nhận được hồ sơ, Cơ quan thẩm quyền quản lý CITES Việt Nam thông báo cho tổ chức, cá nhân biết.\r',
 'Cách thức thực hiện:\r',
 '|Hình thức nộp|Thời hạn giải quyết|Phí, lệ phí|Mô tả|\r',
 '|--|--|--|-|\r',
 '

## Generate Question từ các context trên với prompt sau:
```
prompt3 = """
Bạn là một AI được giao nhiệm vụ chuyển đổi văn bản về thủ tục dịch vụ công thành bộ câu hỏi và trả lời TIẾNG VIỆT.

Hướng dẫn:
1. Nguồn thông tin: Cả câu hỏi và câu trả lời phải được trích xuất trực tiếp từ văn bản đã cho. Không suy đoán hoặc thêm thông tin ngoài những gì có trong văn bản.
2. Yêu cầu về câu hỏi:
-Không hỏi về tên của thủ tục.
-Mỗi câu hỏi phải bao gồm "Tên thủ tục" cụ thể có trong văn bản.
-Không được sử dụng các cụm từ chung chung như "thủ tục này", "trong văn bản", "thủ tục nào".
-Các câu hỏi phải chính xác và liên quan trực tiếp đến thông tin được cung cấp. Tránh hỏi những câu hỏi không rõ ràng hoặc không liên quan.
-Đảm bảo mỗi câu hỏi khác nhau về cấu trúc nhưng vẫn hỏi về những khía cạnh cụ thể liên quan đến thủ tục được đề cập.
3. Yêu cầu về câu trả lời:
-Câu trả lời phải đầy đủ, chính xác và được trích dẫn trực tiếp từ văn bản.
-Trả lời một cách rõ ràng và không mơ hồ, không sử dụng các đại từ chung chung như "nó", "điều này", v.v.
4.Đa dạng câu hỏi: Câu hỏi phải có tính đa dạng về cách diễn đạt nhưng vẫn giữ được sự nhất quán về nội dung thông tin. Dùng các từ đồng nghĩa hoặc cách diễn đạt khác nhau để hỏi về các khía cạnh khác nhau của thủ tục.
5.Số lượng câu hỏi: Kết quả phải bao gồm số lượng câu hỏi và câu trả lời được yêu cầu.

Ví dụ về câu hỏi sai:
Tên của thủ tục này là gì?
Cấp thực hiện của thủ tục này là gì?
Loại thủ tục của thủ tục này là gì ?


Kết quả với 2 câu hỏi và trả lời:
[Q]: Thủ tục báo cáo giải trình nhu cầu, thay đổi nhu cầu sử dụng người lao động nước ngoài thuộc loại thủ tục gì?
[A]: Thủ tục báo cáo giải trình nhu cầu, thay đổi nhu cầu sử dụng người lao động nước ngoài thuộc loại thủ tục TTHC được luật giao quy định chi tiết.
[Q]: Thủ tục báo cáo giải trình nhu cầu, thay đổi nhu cầu sử dụng người lao động nước ngoài có mã là gì?
[A]: Mã thủ tục của thủ tục báo cáo giải trình nhu cầu, thay đổi nhu cầu sử dụng người lao động nước ngoài là 1.000105.

Văn bản:

{{text}}

Kết quả với {{num_questions}} câu hỏi và trả lời:

"""
```


## Đọc các QA dataframe từ 3 model (Mistral, Qwen và LLama3) <br>
Số lượng context: 100 <br>
Số lượng Question mỗi context: 5

$\Longrightarrow$ Tổng số lượng câu hỏi mỗi dataframe: 5*100 = 500

In [13]:
qa_llama3_df = pd.read_parquet('./data/llm_question_gen/qa_llama3.parquet')
print('Size of QA Llama3 dataframe: ', len(qa_llama3_df))

qa_mistral_df = pd.read_parquet('./data/llm_question_gen/qa_mistral.parquet')
print('Size of QA Mitsral dataframe: ', len(qa_mistral_df))

qa_qwen_df = pd.read_parquet('./data/llm_question_gen/qa_qwen.parquet')
print('Size of QA Qwen dataframe: ', len(qa_qwen_df))

Size of QA Llama3 dataframe:  500
Size of QA Mitsral dataframe:  500
Size of QA Qwen dataframe:  500


In [12]:
qa_llama3_df.head()

,retrieval_gt,qid,query,generation_gt
0,[[b4597feb-b61e-405d-b270-89b3bba0706f]],6d277cde-c152-412f-8bff-c05d5abd455c,Thủ tục Cấp ấn phẩm Chứng chỉ CITES xuất khẩu ...,[Thủ tục Cấp ấn phẩm Chứng chỉ CITES xuất khẩu...
1,[[b4597feb-b61e-405d-b270-89b3bba0706f]],28aaa2d1-7e16-4143-bf55-872ad6254b7b,Thủ tục Cấp ấn phẩm Chứng chỉ CITES xuất khẩu ...,[Mã thủ tục của thủ tục Cấp ấn phẩm Chứng chỉ ...
2,[[b4597feb-b61e-405d-b270-89b3bba0706f]],dd4d482b-3b0d-4909-aee1-93002fa061da,Thủ tục Cấp ấn phẩm Chứng chỉ CITES xuất khẩu ...,[Thủ tục Cấp ấn phẩm Chứng chỉ CITES xuất khẩu...
3,[[b4597feb-b61e-405d-b270-89b3bba0706f]],81ecc484-0b20-4211-9d19-32cdf8323529,Thủ tục Cấp ấn phẩm Chứng chỉ CITES xuất khẩu ...,[Thủ tục Cấp ấn phẩm Chứng chỉ CITES xuất khẩu...
4,[[b4597feb-b61e-405d-b270-89b3bba0706f]],48e4c189-e074-4387-a67c-b7c1fe1e41ef,Thời gian giải quyết thủ tục Cấp ấn phẩm Chứng...,[Thời gian giải quyết thủ tục Cấp ấn phẩm Chứn...


## Tiến hành cross-checking với mỗi model. Ví dụ với LLama3: <br>
```
verified_prompt = """Bạn sẽ được cung cấp một đoạn văn bản và 5 câu hỏi. Hãy xác nhận liệu các câu hỏi có liên quan đến đoạn văn bản này không. Nếu câu hỏi có liên quan, hãy cung cấp câu trả lời dựa trên thông tin trong context. Nếu câu hỏi không liên quan, phải trả lời "Không".

Yêu cầu:
Đọc kỹ đoạn văn bản và câu hỏi.
Xác định liệu câu hỏi có liên quan trực tiếp đến nội dung của đoạn văn bản hay không.
Nếu câu hỏi có liên quan, hãy trả lời câu hỏi dựa trên thông tin từ đoạn văn bản, trả lời một cách đầy đủ, có chủ ngữ.
Nếu câu hỏi có liên quan, không trả ra câu hỏi, chỉ cần trả ra câu trả lời.
Sau mỗi câu trả lời đều phải có thêm kí tự '[END]'.
Nếu câu hỏi không liên quan, hãy trả lời "Không".
Nếu câu hỏi không cung cấp tên văn bản cụ thể, hãy trả lời là "Không".
Nếu bạn không biết trả lời, hãy trả lời "Không".
Số lượng câu trả lời luôn luôn phải là 5.

Đoạn văn bản:
Chi tiết thủ tục hành chính:
Mã thủ tục:
1.000105
Số quyết định:
1560/QĐ-LĐTBXH
Tên thủ tục:
Báo cáo giải trình nhu cầu, thay đổi nhu cầu sử dụng người lao động nước ngoài
Cấp thực hiện:
Cấp Tỉnh
Loại thủ tục:
TTHC được luật giao quy định chi tiết
Lĩnh vực:
Việc làm

5 câu hỏi:
Thủ tục báo cáo giải trình nhu cầu, thay đổi nhu cầu sử dụng người lao động nước ngoài thuộc loại thủ tục gì?
Mã thủ tục báo cáo giải trình nhu cầu, thay đổi nhu cầu sử dụng người lao động nước ngoài là gì?
Thủ tục trên được thực hiện ở đâu?
Văn bản trên có mã thủ tục là gì?
Lĩnh vực thực hiện của báo cáo giải trình nhu cầu, thay đổi nhu cầu sử dụng người lao động nước ngoài là gì?

5 câu trả lời:
Thủ tục báo cáo giải trình nhu cầu, thay đổi nhu cầu sử dụng người lao động nước ngoài thuộc loại thủ tục TTHC được luật giao quy định chi tiết. [END]
Mã thủ tục của thủ tục báo cáo giải trình nhu cầu, thay đổi nhu cầu sử dụng người lao động nước ngoài là 1.000105. [END]
Không [END]
Không [END]
Lĩnh vực thực hiện của báo cáo giải trình nhu cầu, thay đổi nhu cầu sử dụng người lao động nước ngoài là Việc làm. [END]

Đoạn văn bản:
{context}
5 câu hỏi:
{questions}
5 câu trả lời:
"""

class CrossCheckingLLMs:
def __init__(
    self,
    input_dataframe: pd.DataFrame,
    qa_dataframe: pd.DataFrame,
    generator_LLM: str,
    checker_LLM: List[str]
)
checker = CrossCheckingLLMs(corpus_df[0:100], llama3_df, llm_llama3, [llm_qwen, llm_mistral])
``` 

In [39]:
qa_question_verified_llama3_df = pd.read_csv('./data/llm_verified_question/llama3_verified_dataframe_500.csv')
print('Size of Verified questions of Llama3 dataframe: ', len(qa_question_verified_llama3_df))

Size of Verified questions of Llama3 dataframe:  497


In [40]:
qa_question_verified_llama3_df.head()

,Context,Question,Answers
0,Chi tiết thủ tục hành chính:\r\nMã thủ tục:\r\...,Thủ tục Cấp ấn phẩm Chứng chỉ CITES xuất khẩu ...,['Thủ tục Cấp ấn phẩm Chứng chỉ CITES xuất khẩ...
1,Chi tiết thủ tục hành chính:\r\nMã thủ tục:\r\...,Thủ tục Cấp ấn phẩm Chứng chỉ CITES xuất khẩu ...,['Mã thủ tục của thủ tục Cấp ấn phẩm Chứng chỉ...
2,Chi tiết thủ tục hành chính:\r\nMã thủ tục:\r\...,Thủ tục Cấp ấn phẩm Chứng chỉ CITES xuất khẩu ...,['Thủ tục Cấp ấn phẩm Chứng chỉ CITES xuất khẩ...
3,Chi tiết thủ tục hành chính:\r\nMã thủ tục:\r\...,Thủ tục Cấp ấn phẩm Chứng chỉ CITES xuất khẩu ...,['Thủ tục Cấp ấn phẩm Chứng chỉ CITES xuất khẩ...
4,Chi tiết thủ tục hành chính:\r\nMã thủ tục:\r\...,Thời gian giải quyết thủ tục Cấp ấn phẩm Chứng...,['Thời gian giải quyết thủ tục Cấp ấn phẩm Chứ...


In [41]:
qa_question_verified_llama3_df['Answers'][2].split(',')

["['Thủ tục Cấp ấn phẩm Chứng chỉ CITES xuất khẩu mẫu vật lưu niệm được thực hiện bởi Cơ quan Quản lý CITES Việt Nam - Bộ NN-PTNT.'",
 " 'Thủ tục Cấp ấn phẩm Chứng chỉ CITES xuất khẩu mẫu vật lưu niệm được thực hiện bởi Cơ quan Quản lý CITES Việt Nam - Bộ NN-PTNT.'",
 " 'Thủ tục Cấp ấn phẩm Chứng chỉ CITES xuất khẩu mẫu vật lưu niệm được thực hiện bởi Cấp Bộ.']"]

## Kiểm tra câu trả lời
Sau khi đã đảm bảo được các question có liên quan đến context, tiến hành kiểm tra xem liệu các answer từ các LLM có đồng nhất hay không?<br>
Ví dụ với Dataframe chứa question đã được verified, tạo ra từ LLama3:
```
class SimilarityCheck:
    def __init__(
        self,
        input_dataframe: pd.DataFrame,
        similarity_model: str,
        output_path: str,
    ):
model = "dangvantuan/vietnamese-embedding-LongContext"
sim_checker = SimilarityCheck(qa_verified_llama3_df,model,output_path='llama3_sim_dataframe.csv')
```

In [45]:
qa_answer_verified_llama3_df = pd.read_csv('data/llm_verifed_answer/llama3_sim_dataframe.csv')
print('Size of Verified answer of Llama3 dataframe: ', len(qa_answer_verified_llama3_df))

Size of Verified answer of Llama3 dataframe:  452


In [47]:
qa_answer_verified_llama3_df.head()

,Context,Question,Answers,avg_similarity
0,Chi tiết thủ tục hành chính:\r\nMã thủ tục:\r\...,Thủ tục Cấp ấn phẩm Chứng chỉ CITES xuất khẩu ...,['Thủ tục Cấp ấn phẩm Chứng chỉ CITES xuất khẩ...,1.000000
1,Chi tiết thủ tục hành chính:\r\nMã thủ tục:\r\...,Thủ tục Cấp ấn phẩm Chứng chỉ CITES xuất khẩu ...,['Mã thủ tục của thủ tục Cấp ấn phẩm Chứng chỉ...,1.000000
2,Chi tiết thủ tục hành chính:\r\nMã thủ tục:\r\...,Thủ tục Cấp ấn phẩm Chứng chỉ CITES xuất khẩu ...,['Thủ tục Cấp ấn phẩm Chứng chỉ CITES xuất khẩ...,0.943554
3,Chi tiết thủ tục hành chính:\r\nMã thủ tục:\r\...,Thủ tục Cấp ấn phẩm Chứng chỉ CITES xuất khẩu ...,['Thủ tục Cấp ấn phẩm Chứng chỉ CITES xuất khẩ...,0.989094
4,Chi tiết thủ tục hành chính:\r\nMã thủ tục:\r\...,Thời gian giải quyết thủ tục Cấp ấn phẩm Chứng...,['Thời gian giải quyết thủ tục Cấp ấn phẩm Chứ...,0.985785


In [50]:
qa_answer_verified_llama3_df['Answers'][2].split(',')

["['Thủ tục Cấp ấn phẩm Chứng chỉ CITES xuất khẩu mẫu vật lưu niệm được thực hiện bởi Cơ quan Quản lý CITES Việt Nam - Bộ NN-PTNT.'",
 " 'Thủ tục Cấp ấn phẩm Chứng chỉ CITES xuất khẩu mẫu vật lưu niệm được thực hiện bởi Cơ quan Quản lý CITES Việt Nam - Bộ NN-PTNT.'",
 " 'Thủ tục Cấp ấn phẩm Chứng chỉ CITES xuất khẩu mẫu vật lưu niệm được thực hiện bởi Cấp Bộ.']"]

## Ghép các dataframe lại với nhau, cho ra kết quả cuối cùng

In [51]:
final_df = pd.read_csv('data/final_dataset/100_verified_context_QA.csv')
print('Size of final dataframe: ', len(final_df))

Size of final dataframe:  883


In [52]:
final_df.head()

,Context,Question,Answers
0,Chi tiết thủ tục hành chính:\r\nMã thủ tục:\r\...,"Cơ quan nào thực hiện thủ tục ""Cấp ấn phẩm Chứ...","['Thủ tục ""Cấp ấn phẩm Chứng chỉ CITES xuất kh..."
1,Chi tiết thủ tục hành chính:\r\nMã thủ tục:\r\...,Thành phần hồ sơ cần thiết để thực hiện thủ tụ...,"['Hồ sơ cần thiết để thực hiện thủ tục ""Cấp ấn..."
2,Chi tiết thủ tục hành chính:\r\nMã thủ tục:\r\...,Thời gian giải quyết thủ tục Cấp ấn phẩm Chứng...,['Thời gian giải quyết thủ tục Cấp ấn phẩm Chứ...
3,Chi tiết thủ tục hành chính:\r\nMã thủ tục:\r\...,"Thủ tục ""Cấp ấn phẩm Chứng chỉ CITES xuất khẩu...","['Thủ tục ""Cấp ấn phẩm Chứng chỉ CITES xuất kh..."
4,Chi tiết thủ tục hành chính:\r\nMã thủ tục:\r\...,"Thủ tục ""Cấp ấn phẩm Chứng chỉ CITES xuất khẩu...","['Thủ tục ""Cấp ấn phẩm Chứng chỉ CITES xuất kh..."


In [58]:
final_df['Question'][6]

'Thủ tục Cấp ấn phẩm Chứng chỉ CITES xuất khẩu mẫu vật lưu niệm có thể được thực hiện theo hình thức nào?'

In [61]:
ast.literal_eval(final_df['Answers'][6])

['Thủ tục Cấp ấn phẩm Chứng chỉ CITES xuất khẩu mẫu vật lưu niệm có thể được thực hiện theo hình thức Trực tiếp, Trực tuyến, Dịch vụ bưu chính.',
 'Thủ tục Cấp ấn phẩm Chứng chỉ CITES xuất khẩu mẫu vật lưu niệm có thể được thực hiện theo hình thức trực tiếp, trực tuyến hoặc qua dịch vụ bưu chính.',
 'Thủ tục Cấp ấn phẩm Chứng chỉ CITES xuất khẩu mẫu vật lưu niệm có thể được thực hiện theo các hình thức trực tiếp, trực tuyến và dịch vụ bưu chính.']